In [1]:
import os
import pandas as pd
from textblob import *
import nltk
import numpy as np
from sklearn import tree
import openpyxl
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


training = pd.read_csv(r"data/imdb.csv", encoding="utf8")


#remove nonASCIICharacters
training["text"] = training.apply(lambda x: "".join([i for i in x["text"] if ord(i) <= 127]), axis=1)
training






,text,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [2]:
def to_class(s):
    match s:
        case "positive":
            return 1
        case "negative":
            return 0

In [3]:
training["sentiment"] = training.apply(lambda x: to_class(x["sentiment"]), axis=1)

In [4]:


training["text_tb_pol"] = training.apply(lambda x: TextBlob(x["text"]).polarity, axis=1)
training["text_tb_sub"] = training.apply(lambda x: TextBlob(x["text"]).subjectivity, axis=1)





In [8]:
analyzer = SentimentIntensityAnalyzer()

training["text_vader_scores"] = training.apply(lambda x: analyzer.polarity_scores(x["text"]), axis=1)
training["text_vader_comp"] = training.apply(lambda x: x["text_vader_scores"]["compound"], axis=1)
training["text_vader_neg"] = training.apply(lambda x: x["text_vader_scores"]["neg"], axis=1)
training["text_vader_neu"] = training.apply(lambda x: x["text_vader_scores"]["neu"], axis=1)
training["text_vader_pos"] = training.apply(lambda x: x["text_vader_scores"]["pos"], axis=1)
training = training.drop(["text_vader_scores"], axis=1)








In [10]:
training

,text,sentiment,text_tb_pol,text_tb_sub,text_vader_neg,text_vader_neu,text_vader_pos,text_vader_comp
0,One of the other reviewers has mentioned that ...,1,0.023433,0.490369,0.179,0.756,0.064,-0.9916
1,A wonderful little production. <br /><br />The...,1,0.109722,0.559343,0.052,0.773,0.176,0.9670
2,I thought this was a wonderful way to spend ti...,1,0.354008,0.658730,0.114,0.688,0.198,0.9519
3,Basically there's a family where a little boy ...,0,-0.057813,0.454167,0.125,0.816,0.059,-0.9213
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,0.217952,0.452916,0.050,0.806,0.144,0.9744
...,...,...,...,...,...,...,...,...
49995,I thought this movie did a down right good job...,1,0.394425,0.522897,0.045,0.765,0.189,0.9886
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0,-0.276190,0.642857,0.160,0.730,0.110,-0.6837
49997,I am a Catholic taught in parochial elementary...,0,0.056984,0.395538,0.181,0.704,0.115,-0.9734
49998,I'm going to have to disagree with the previou...,0,-0.048663,0.447034,0.116,0.804,0.080,-0.8657


In [ ]:
'''
    Regression, let's see if TextBlob and Vader are correlated
'''
from sklearn.linear_model import LinearRegression

y = training["text_tb_pol"].to_numpy()
x = training[["text_vader_comp", "text_vader_neg", "text_vader_pos", "text_vader_neu"]].to_numpy()
#x = training["text_vader_neu"].to_numpy().reshape(-1, 1)
reg = LinearRegression().fit(x, y)
reg.coef_




array([ 0.00282125, -0.48287496,  1.87121906,  0.81874365])

In [ ]:
'''
    Begin logistic regression
'''
'''
solver          penalty
lbfgs           l2
newton-cg       l1, l2
sag             l2
saga            elasticnet, l1, l2


'''

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

classes = training["sentiment"].to_numpy()
#features = training[["text_tb_pol", "text_tb_sub", "hash_tb_pol", "hash_tb_sub", "text_vader_neg", "text_vader_neu", "text_vader_pos", "hash_vader_neg", "hash_vader_neu", "hash_vader_pos"]].to_numpy()

'''text only'''
features = training[["text_tb_pol",  "text_vader_pos", "text_vader_neg", "text_vader_neu"]].to_numpy()

#validate_features = testing[["text_tb_pol", "text_tb_sub", "hash_tb_pol", "hash_tb_sub", "text_vader_neg", "text_vader_neu", "text_vader_pos", "hash_vader_neg", "hash_vader_neu", "hash_vader_pos"]].to_numpy()


#for i in range(100):

averages = []

total = 0
length = 0

#lbgfs
for i in range(100):
    x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=i)
    lbgfs = LogisticRegression(penalty="l2", solver="lbfgs")
    lbgfs.fit(x_train, y_train)
    results = lbgfs.predict(x_test)
    actual = y_test
    total += (len([i for i in zip(results, actual) if i[0] == i[1]]) / len(actual))
    length += 1
averages.append(total / length)

''''''
total = 0
length = 0
#newton-cg
for i in range(100):
    x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=i)
    newton_cg = LogisticRegression(penalty="l2", solver="newton-cg")
    newton_cg.fit(x_train, y_train)
    results = newton_cg.predict(x_test)
    actual = y_test
    total += (len([i for i in zip(results, actual) if i[0] == i[1]]) / len(actual))
    length += 1
averages.append(total / length)

total = 0
length = 0
#sag
for i in range(100):
    x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=i)
    sag = LogisticRegression(penalty="l2", solver="sag", random_state=i)
    sag.fit(x_train, y_train)
    results = sag.predict(x_test)
    actual = y_test
    total += (len([i for i in zip(results, actual) if i[0] == i[1]]) / len(actual))
    length += 1
averages.append(total / length)

total = 0
length = 0
#saga
for i in range(100):
    x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=i)
    saga = LogisticRegression(penalty="l2", solver="saga", random_state=i)
    saga.fit(x_train, y_train)
    results = saga.predict(x_test)
    actual = y_test
    total += (len([i for i in zip(results, actual) if i[0] == i[1]]) / len(actual))
    length += 1
averages.append(total / length)


averages





[0.772304, 0.7723013333333333, 0.7723186666666666, 0.7723093333333334]

In [ ]:
from sklearn.metrics import confusion_matrix
matrices = []
for i in range(100):
    x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=i)
    lbgfs = LogisticRegression(penalty="l2", solver="lbfgs")
    lbgfs.fit(x_train, y_train)
    results = lbgfs.predict(x_test)
    actual = y_test
    matrices.append(confusion_matrix(y_test, results))

mean_matrix = np.mean(np.array(matrices), axis=0)
mean_matrix

array([[395.66,   8.36,  27.09],
       [ 81.47,   7.91,   2.13],
       [167.73,   2.01,  34.64]])